In [1]:
epochs = 10
batch_size = 256

train_dir_path = 'train/simpsons_dataset'
test_dir_path = 'testset/testset'

state_path = 'output/state.pth'
history_path = 'output/history.pkl'

In [2]:
import os

os.environ["MASTER_ADDR"] = "127.0.0.1"
os.environ["MASTER_PORT"] = "8888"

In [3]:
from pathlib import Path
import torch.multiprocessing as mp
from src.train import train
from sklearn.preprocessing import LabelEncoder
import torch

TRAIN_DIR = Path(train_dir_path)
TEST_DIR = Path(test_dir_path)

train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

label_encoder = LabelEncoder()
label_encoder.fit([path.parent.name for path in train_val_files])
gpu_count = torch.cuda.device_count()

In [4]:
%%time
mp.spawn(train, nprocs=gpu_count, args=(train_val_files, gpu_count, batch_size, epochs, label_encoder, state_path, history_path))

W0509 15:46:31.574000 140224959033344 torch/multiprocessing/spawn.py:145] Terminating process 130951 via signal SIGTERM


ProcessRaisedException: 

-- Process 1 terminated with the following error:
Traceback (most recent call last):
  File "/home/brotiger/.local/lib/python3.10/site-packages/torch/multiprocessing/spawn.py", line 75, in _wrap
    fn(i, *args)
  File "/home/brotiger/projects/mlstart/simpsons_cnn/src/train.py", line 75, in train
    val_dataset = SimpsonsDataset(files, label_encoder, mode='val')
NameError: name 'files' is not defined


In [5]:
import pickle

with open(history_path, 'rb') as f:
    history = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'output/history.pkl'

In [ ]:
for history_key in history:
    plt.figure(figsize=(15, 9))
    plt.plot(history[history_key]["train"], label=f"train_{history_key}")
    plt.plot(history[history_key]["val"], label=f"val_{history_key}")
    plt.legend(loc='best')
    plt.xlabel("epochs")
    plt.ylabel(history_key)
    plt.show()